In [2]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.tokenize import word_tokenize
import nltk


In [3]:
data = pd.read_csv('../CSV/preped_data.csv')

labels = pd.read_csv('../CSV/labels.csv')


In [4]:
data.shape

(9494, 5)

In [5]:
labels.shape

(220, 1)

In [6]:

data['merged'] = data['description'].fillna('') + ' ' + data['business_tags'].fillna('') + ' ' + data['sector'].fillna('') + ' ' + data['category'].fillna('') + ' ' + data['niche'].fillna('')

datas = [TaggedDocument(words = word_tokenize(text), tags = [str(i)]) for i, text in enumerate(data['merged'])]

model = Doc2Vec(datas, vector_size = 4500, window = 5, min_count = 2, workers = 4, dm=0)

In [7]:
data['embedding'] = data['merged'].apply(lambda x: model.infer_vector(word_tokenize(x)))

labels['embedding'] = labels['label'].apply(lambda x: model.infer_vector(word_tokenize(x)))

In [8]:
data_embeddings = list(data['embedding'])
label_embeddings = list(labels['embedding'])

cosine_similarities = cosine_similarity(data_embeddings, label_embeddings)

In [9]:
def get_top_labels(similarities, labels, top_n=3):
    top_indices = similarities.argsort()[:, -top_n:][:, ::-1]
    top_labels = [[labels[i] for i in indices] for indices in top_indices]
    top_labels_with_scores = [[(labels[i], similarities[row_idx, i]) for i in indices] for row_idx, indices in enumerate(top_indices)]
    return top_labels_with_scores

data['top_labels'] = get_top_labels(cosine_similarities, labels['label'])

# Display the results
output_path = 'c:/Users/anama_1lv/Desktop/Veridion/Company_Classifier/CSV/company_top_labels_dbow.csv'
data[['description', 'business_tags', 'sector', 'category', 'niche', 'top_labels']].to_csv(output_path, index=False)